# Skript for Live Vehicle Data Cleaning - Part 3

<strong><em>Important: This is a guide, which helps and explains you the data cleaning we where doing before this Hack-a-thon. There are parts you can and sometimes should directly copy and paste. You won't be able to copy the whole notebook and run it within your project.</em></strong>

## Creating the Client Connection to the Cloud Object Storage and the "smart-city-live-vehicle-positions" bucket

The following code cell can be automatically inserted trough the Notebook UI. To do so, click on the data button (top right corner) there you find the *files* and *connections* tab. Go to the *connection* as we want to create a client to our Cloud Object Storage. 

There you will find the Connection which we created before. Click "insert to code" and choose the "StreamingBody object" option. After that there will open a pop up which showes you the folder structure of your underlying cloud bucket. Choose the right folders and subfolders until you end up in the last subfolder, that contains all the .json files we need. Choose one file and click *Select*. Next you will see a code cell, inserted automatically, that looks like this one except it contains the correct api-keys etc.

> It doesn't matter which .json you will choose, because we will later on only use the created client object to access more then only one .json file.

## Converting the tst into a other timestamp format 

We want to do this, because
1. SPSS Modeller don't understand the given format in `tst`
2. We maybe want to enrich the data by JOIN it with another resource.
   
So we define a method, which takes every `tst` value and save a changed version of that into a list, which becomes a new column after that.

In [ ]:
import datetime

date = []
hour = []

for x in df_lvd_cleaned.tst:
    x = x[:-11]
    date_obj = datetime.datetime.strptime(x, '%Y-%m-%dT%H')
    date.append(str(date_obj.date()) + " " + str(date_obj.time()))
    
date

In [20]:
df_lvd_cleaned['timestamp'] = date

# Save the data back to our project

If we think we are finished with cleaning our data, which doesn't have to mean you have the exact same result, we want to extract the data out of the notebook and back into our project space. There we can use it as a data asset. 

To do so, we use the python libary `project_lib` and import `Project` from it. This gives us the needed functionality, to save the data (dataFrame is converted trough a pandas.DataFrame method named `to_csv()` into a csv format) back to our project space where it can be found as an data asset. 

In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='id', project_access_token='access-token')
pc = project.project_context

project.save_data(data=df_lvd_cleaned.to_csv(index=False),file_name=str(dateloading)+'_only_lvd.csv',overwrite=True)